**UNIVERSIDADE FEDERAL DE PERNAMBUCO.
CENTRO DE INFORMÁTICA.
PROGRAMA DE PÓS-GRADUAÇÃO EM CIÊNCIA DA COMPUTAÇÃO.
DISCIPLINA: APRENDIZAGEM DE MÁQUINA**

Luiza Carvalho Silveira (lcs11@cin.ufpe.br),
Andersson Alves da Silva (aas9@cin.ufpe.br)

**Missão 04**.  Árvores de Decisão - Como a Multicolinearidade afeta as árvores de decisão e quais as estratégias para contornar possíveis problemas?

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Carregando os dados

data = pd.read_csv('/content/gdrive/My Drive/data/Diamonds.csv', sep =",")

data.head()

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,326,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,327,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,334,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,335,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


In [ ]:
data.cut.unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [ ]:
# Mapeando os dados do corte do diamante

cut_mapping = {
               'Ideal': 4,
               'Premium': 3,
               'Very Good': 2,
               'Good': 1,
               'Fair': 0}


data['cut'] = data['cut'].map(cut_mapping)

data.head()

,price,carat,cut,color,clarity,depth,table,x,y,z
0,326,0.23,4,E,SI2,61.5,55.0,3.95,3.98,2.43
1,326,0.21,3,E,SI1,59.8,61.0,3.89,3.84,2.31
2,327,0.23,1,E,VS1,56.9,65.0,4.05,4.07,2.31
3,334,0.29,3,I,VS2,62.4,58.0,4.20,4.23,2.63
4,335,0.31,1,J,SI2,63.3,58.0,4.34,4.35,2.75


In [ ]:
data = pd.get_dummies(data) # one hot encoding para dados nominais

In [ ]:
# Plotando Matriz de correlação

corr = data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,price,carat,cut,depth,table,x,y,z,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
price,1.00,0.92,-0.05,-0.01,0.13,0.88,0.87,0.86,-0.07,-0.10,-0.02,0.01,0.06,0.10,0.08,-0.00,-0.05,0.01,0.13,-0.01,-0.00,-0.10,-0.05
carat,0.92,1.00,-0.13,0.03,0.18,0.98,0.95,0.95,-0.11,-0.14,-0.06,-0.03,0.10,0.16,0.18,0.12,-0.11,0.06,0.27,-0.06,-0.04,-0.17,-0.14
cut,-0.05,-0.13,1.00,-0.22,-0.43,-0.13,-0.12,-0.15,0.01,-0.00,-0.01,0.04,-0.01,-0.01,-0.04,-0.10,0.09,-0.08,-0.10,0.04,0.04,0.08,0.06
depth,-0.01,0.03,-0.22,1.00,-0.30,-0.03,-0.03,0.09,-0.01,-0.03,-0.02,0.00,0.03,0.02,0.02,0.08,-0.03,0.04,0.01,-0.02,-0.01,-0.02,-0.02
table,0.13,0.18,-0.43,-0.30,1.00,0.20,0.18,0.15,-0.01,0.01,-0.00,-0.04,0.01,0.02,0.04,0.04,-0.08,0.05,0.10,-0.03,-0.01,-0.07,-0.06
x,0.88,0.98,-0.13,-0.03,0.20,1.00,0.97,0.97,-0.11,-0.13,-0.05,-0.02,0.10,0.15,0.16,0.11,-0.13,0.08,0.27,-0.06,-0.04,-0.19,-0.15
y,0.87,0.95,-0.12,-0.03,0.18,0.97,1.00,0.95,-0.10,-0.13,-0.05,-0.02,0.09,0.14,0.16,0.10,-0.12,0.08,0.26,-0.06,-0.04,-0.18,-0.14
z,0.86,0.95,-0.15,0.09,0.15,0.97,0.95,1.00,-0.11,-0.13,-0.05,-0.02,0.10,0.15,0.16,0.11,-0.13,0.08,0.26,-0.06,-0.04,-0.18,-0.14
color_D,-0.07,-0.11,0.01,-0.01,-0.01,-0.11,-0.10,-0.11,1.00,-0.18,-0.18,-0.20,-0.16,-0.13,-0.09,-0.02,-0.05,0.06,0.03,-0.05,0.02,-0.05,-0.02
color_E,-0.10,-0.14,-0.00,-0.03,0.01,-0.13,-0.13,-0.13,-0.18,1.00,-0.22,-0.24,-0.20,-0.16,-0.11,-0.01,-0.04,0.01,0.01,-0.03,0.03,-0.00,0.01


Pelo gráfico da correlação das variáveis é visível que x, y e z tem correlação muito alta entre si e com a variável carat.

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Criar uma regressão de carat usando as outras features
y, X = dmatrices("carat ~  depth + table + price + x + y + z", data, return_type = 'dataframe')

# Para cada X calcular o VIF e salvar no dataframe
VIF = pd.DataFrame()
VIF["Features"] = X.columns
VIF["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(VIF)

    Features          VIF
0  Intercept  4105.462403
1      depth     1.432522
2      table     1.152410
3      price     4.642329
4          x    40.320259
5          y    20.449150
6          z    23.530217


Os valores altos de VIF confirmam a multicolinearidade das varáveis x, y e z

In [ ]:
# Separando a variável de predição do resto do Dataset
y = data["cut"]
X = data.drop(["cut"], axis =1)

data_dmatrix = xgb.DMatrix(data=X,label=y)

# Retirando as variáveis com alta correlação entre si x, y e z
X_alt = data.drop(["cut", "x", "y", "z"], axis = 1)

In [ ]:
# conjuntos de treino e teste COM variáveis de alta correlação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101) 

In [ ]:
# conjuntos de treino e teste SEM variáveis de alta correlação
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_alt, y, test_size=0.30, random_state=101) 

## Árvore de decisão com xgboost

In [ ]:
# Árvore de decisão sem o cross validation para visualização da saída de maneira mais simples

xg_class = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, num_class = 5)

xg_class.fit(X_train,y_train)

preds = xg_class.predict(X_test)

print(classification_report(y_test,preds))
print("F1 Score")
print(metrics.f1_score(y_test, preds, average='weighted', labels=np.unique(y_test)))

              precision    recall  f1-score   support

           0       0.87      0.51      0.64       486
           1       0.84      0.48      0.61      1477
           2       0.64      0.32      0.43      3626
           3       0.64      0.80      0.71      4155
           4       0.76      0.94      0.84      6438

    accuracy                           0.71     16182
   macro avg       0.75      0.61      0.64     16182
weighted avg       0.71      0.71      0.69     16182

F1 Score
0.6862167299850146


In [ ]:
# Sem utilizar os Dados com Alta Correlação

xg_class1 = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, num_class = 5)

xg_class1.fit(X_train_a,y_train_a)

preds1 = xg_class1.predict(X_test_a)

print(classification_report(y_test_a,preds1))
print("F1 Score")
print(metrics.f1_score(y_test_a, preds1, average='weighted', labels=np.unique(y_test_a)))

              precision    recall  f1-score   support

           0       0.91      0.02      0.04       486
           1       0.80      0.11      0.20      1477
           2       0.62      0.33      0.43      3626
           3       0.59      0.77      0.67      4155
           4       0.71      0.95      0.81      6438

    accuracy                           0.66     16182
   macro avg       0.73      0.44      0.43     16182
weighted avg       0.67      0.66      0.61     16182

F1 Score
0.6104348839271938


A presença de multicolinearidade não preocupa as árvores de decisão. A tentativa de remover a multicolinearidade resultou em uma baixa no desempenho da árvore de classificação.

## Regressão logística com SKLearn

In [ ]:
# Regressão Logística COM as variáveis de alta correlação

logmodel = LogisticRegression(max_iter=800)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))
print("F1 Score")
print(metrics.f1_score(y_test, predictions, average='weighted', labels=np.unique(y_test)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       486
           1       0.00      0.00      0.00      1477
           2       0.00      0.00      0.00      3626
           3       0.52      0.67      0.58      4155
           4       0.56      0.93      0.70      6438

    accuracy                           0.54     16182
   macro avg       0.21      0.32      0.26     16182
weighted avg       0.35      0.54      0.43     16182

F1 Score
0.4264942462468031


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Regressão Logística SEM as Variáveis de Alta Correlação

logmodel1 = LogisticRegression(max_iter=800)
logmodel1.fit(X_train_a,y_train_a)
predictions1 = logmodel1.predict(X_test_a)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
print(classification_report(y_test_a,predictions1))
print("F1 Score")
print(metrics.f1_score(y_test_a, predictions1, average='weighted', labels=np.unique(y_test_a)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       486
           1       0.00      0.00      0.00      1477
           2       0.28      0.01      0.02      3626
           3       0.52      0.72      0.60      4155
           4       0.57      0.91      0.70      6438

    accuracy                           0.55     16182
   macro avg       0.27      0.33      0.26     16182
weighted avg       0.42      0.55      0.44     16182

F1 Score
0.43838599863592037


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Houve um pequeno aumento de performance ao se retirar as variáveis de alta correlação do modelo de regressão, o que mostra que ela realmente é afetada pela multicolinearidade
